In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import cupy as cp
from sklearnex import patch_sklearn

In [2]:
def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    
    return data

def mainBareBones():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Replace NaNs for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers
    print("Converting categorical columns to integers...")
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using AdaBoost
    print("\nTraining the AdaBoost model...")
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy (Optimized): {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainBareBones()


Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Training the AdaBoost model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy (Optimized): 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      1908



In [3]:
def optimize_datatypes(data):
    # Downcast numerical columns
    for col in data.select_dtypes(include=['int']):
        cupy_array = cp.array(data[col], dtype=cp.int32)  # Convert to CuPy array
        data[col] = cupy_array.get()  # Convert back to NumPy array
    for col in data.select_dtypes(include=['float']):
        cupy_array = cp.array(data[col], dtype=cp.float32)  # Convert to CuPy array
        data[col] = cupy_array.get()  # Convert back to NumPy array
    
    return data

def mainCuPY():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Replace NaNs for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers
    print("Converting categorical columns to integers...")
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = cp.array(data.drop('Outcome', axis=1).values)  # Convert to CuPy array
    y = cp.array(data['Outcome'].values)  # Convert to CuPy array
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(cp.asnumpy(X), cp.asnumpy(y), test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using AdaBoost
    print("\nTraining the AdaBoost model...")
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy (Optimized): {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainCuPY()


Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Training the AdaBoost model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy (Optimized): 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      1908



In [4]:
patch_sklearn()  # Apply Intel optimizations

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    
    return data

def mainIntelex():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Replace NaNs for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers
    print("Converting categorical columns to integers...")
    if 'Hypertension' in data.columns:
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Train the model using AdaBoost
    print("\nTraining the AdaBoost model...")
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy (Optimized): {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainIntelex()


Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Training the AdaBoost model...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy (Optimized): 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      1908

